In [1]:
import torch
import torchvision
from torchsummary import summary
from torchvision import transforms
import sys

sys.path.append("/workspaces/ood/")
from scripts_ood.train import train_linear_model
import warnings
import json
from PIL import Image
import torch
from torchvision import transforms
%load_ext autoreload
%autoreload 2
import sys

sys.path.append("/workspaces/ood/")
from pytorch_pretrained_vit import ViT
from scripts_ood.utils import imagenet_sanity_check
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform
import timm

from scripts_ood.utils import load_pretrained_weights_vit
warnings.filterwarnings("ignore")
%load_ext autoreload
%autoreload 2
device = torch.device("cuda")
torch.cuda.is_available()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


True

### 1) Load data

In [2]:
# normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
# transform = transforms.Compose(
#     [
#         transforms.Resize(256),
#         transforms.CenterCrop(224),
#         transforms.ToTensor(),
#         normalize,
#     ]
# )

# vit_base_patch16_384 no linear image net head

In [4]:
model_name = 'vit_base_patch16_384'
model = timm.create_model(model_name, pretrained=True, num_classes=0).to(device)
model.eval();
model_name = 'vit_base_patch16_384_nohead'

In [5]:
config = resolve_data_config({}, model=model)
transform = create_transform(**config)
cifar_data_train = torchvision.datasets.CIFAR10(
    "../data/cifar10", download=False, transform=transform
)
cifar_data_test = torchvision.datasets.CIFAR10(
    "../data/cifar10", download=False, transform=transform, train=False
)

svhn_data_train = torchvision.datasets.SVHN(
    "../data/svhn", download=False, transform=transform
)
svhn_data_test = torchvision.datasets.SVHN(
    "../data/svhn", download=False, transform=transform, split="test"
)

In [6]:
from scripts_ood.utils import add_labels, predict_on_whole_dataset

predict_on_whole_dataset(model, svhn_data_test, f"{model_name}_svhn_test", device)
predict_on_whole_dataset(model, cifar_data_test, f"{model_name}_cifar_test", device)
predict_on_whole_dataset(model, cifar_data_train, f"{model_name}_cifar_train", device)

100%|██████████| 50000/50000 [23:47<00:00, 35.03it/s]


In [7]:
linear_model = train_linear_model(model_name=model_name, device=device, emb_size=768)

epoch: 0, train loss: 2.71451,  train_accuracy: 0.11326, test_accuracy 0.11240
current LR: 0.0001
epoch: 1, train loss: 0.57443,  train_accuracy: 0.95294, test_accuracy 0.95220
current LR: 0.0001
epoch: 2, train loss: 0.47665,  train_accuracy: 0.96866, test_accuracy 0.96870
current LR: 0.0001
epoch: 3, train loss: 0.45657,  train_accuracy: 0.97316, test_accuracy 0.97240
current LR: 0.0001
epoch: 4, train loss: 0.39981,  train_accuracy: 0.97574, test_accuracy 0.97610
current LR: 0.0001
epoch: 5, train loss: 0.39545,  train_accuracy: 0.97746, test_accuracy 0.97720
current LR: 0.0001
epoch: 6, train loss: 0.38562,  train_accuracy: 0.97884, test_accuracy 0.97760
current LR: 0.0001
epoch: 7, train loss: 0.36969,  train_accuracy: 0.97968, test_accuracy 0.97840
current LR: 0.0001
epoch: 8, train loss: 0.36132,  train_accuracy: 0.98052, test_accuracy 0.97880
current LR: 0.0001
epoch: 9, train loss: 0.35402,  train_accuracy: 0.98064, test_accuracy 0.97890
current LR: 0.0001
epoch: 10, train los

In [ ]:
torch.save(linear_model.state_dict(), f'../data/trained_models/{model_name}_cifar_linear_model.pth')

# vit_base_patch16_384 with 1000 head

In [6]:

model_name = 'vit_base_patch16_384'
model = timm.create_model(model_name, pretrained=True, ).to(device)
model.eval();

In [7]:

config = resolve_data_config({}, model=model)
transform = create_transform(**config)

imagenet_sanity_check(model, transform, device)

["282: 'tiger cat'", "281: 'tabby, tabby cat',model_pre", "285: 'Egyptian cat'", "287: 'lynx, catamount'", "288: 'leopard, Panthera pardus'", "292: 'tiger, Panthera tigris'", "290: 'jaguar, panther, Panthera onca, Felis onca'", "289: 'snow leopard, ounce, Panthera uncia'", "284: 'Siamese cat, Siamese'", "286: 'cougar, puma, catamount, mountain lion, painter, panther, Felis concolor'"]


In [9]:
cifar_data_train = torchvision.datasets.CIFAR10(
    "../data/cifar10", download=False, transform=transform
)
cifar_data_test = torchvision.datasets.CIFAR10(
    "../data/cifar10", download=False, transform=transform, train=False
)

svhn_data_train = torchvision.datasets.SVHN(
    "../data/svhn", download=False, transform=transform
)
svhn_data_test = torchvision.datasets.SVHN(
    "../data/svhn", download=False, transform=transform, split="test"
)

In [12]:
from scripts_ood.utils import add_labels, predict_on_whole_dataset

predict_on_whole_dataset(model, svhn_data_test, f"{model_name}_svhn_test", device)
predict_on_whole_dataset(model, cifar_data_test, f"{model_name}_cifar_test", device)
predict_on_whole_dataset(model, cifar_data_train, f"{model_name}_cifar_train", device)

100%|██████████| 50000/50000 [21:58<00:00, 37.92it/s]  


In [13]:
linear_model = train_linear_model(model_name=model_name, device=device, emb_size=1000)

epoch: 0, train loss: 2.66225,  train_accuracy: 0.13706, test_accuracy 0.13350
current LR: 0.0001
epoch: 1, train loss: 0.55760,  train_accuracy: 0.95226, test_accuracy 0.95030
current LR: 0.0001
epoch: 2, train loss: 0.47813,  train_accuracy: 0.96536, test_accuracy 0.96400
current LR: 0.0001
epoch: 3, train loss: 0.42673,  train_accuracy: 0.96996, test_accuracy 0.96760
current LR: 0.0001
epoch: 4, train loss: 0.43435,  train_accuracy: 0.97304, test_accuracy 0.97130
current LR: 0.0001
epoch: 5, train loss: 0.43720,  train_accuracy: 0.97448, test_accuracy 0.97290
current LR: 0.0001
epoch: 6, train loss: 0.39485,  train_accuracy: 0.97592, test_accuracy 0.97370
current LR: 0.0001
epoch: 7, train loss: 0.39622,  train_accuracy: 0.97628, test_accuracy 0.97500
current LR: 0.0001
epoch: 8, train loss: 0.37443,  train_accuracy: 0.97732, test_accuracy 0.97500
current LR: 0.0001
epoch: 9, train loss: 0.34467,  train_accuracy: 0.97800, test_accuracy 0.97520
current LR: 0.0001
epoch: 10, train los

In [ ]:
torch.save(linear_model.state_dict(), f'../data/trained_models/{model_name}_cifar_linear_model.pth')

# Softmax ImageNet resnet50

In [ ]:
soft_model = torchvision.models.resnet50(pretrained=False).to(device)
soft_model.load_state_dict(
    torch.load(
        "/workspaces/ood/data/models/torch/hub/checkpoints/resnet50-0676ba61.pth"
    )
)
summary(soft_model, input_size=(3, 224, 224))

In [ ]:
from ood.utils import imagenet_sanity_check

imagenet_sanity_check(soft_model, transform, device)

In [ ]:
from scripts_ood.utils import add_labels, predict_on_whole_dataset

predict_on_whole_dataset(soft_model, cifar_data_train, "soft_cifar_train")

In [ ]:
model_name = "soft"

linear_model = train_linear_model(model_name=model_name, device=device, emb_size=1000)

In [ ]:
# torch.save(linear_model.state_dict(), f'../data/trained_models/soft_cifar_linear_model0.88test_accuracy.pth')

# MoCo v2 ImageNet pretrain

In [ ]:
from ood.utils import load_moco

model = load_moco("/workspaces/ood/data/models/moco_v2_800ep_pretrain.pth.tar", device)

In [ ]:
from ood.utils import add_labels, predict_on_whole_dataset_moco

predict_on_whole_dataset_moco(model, cifar_data_test, "moco_cifar_test", device)
predict_on_whole_dataset_moco(model, cifar_data_train, "moco_cifar_train", device)
predict_on_whole_dataset_moco(model, svhn_data_train, "moco_svhn_train", device)
predict_on_whole_dataset_moco(model, svhn_data_test, "moco_svhn_test", device)

In [ ]:
model_name = "moco"

linear_model = train_linear_model(model_name=model_name, device=device, emb_size=2048)

In [ ]:
# torch.save(linear_model.state_dict(), f'../data/trained_models/moco_cifar_linear_model0.8382test_accuracy.pth')

# BYOL ImageNet pretrain

In [ ]:
from ood.utils import load_byol

model = load_byol("/workspaces/ood/data/models/pretrain_res50x1.pth.tar", device)

model_name = "byol"
predict_on_whole_dataset(model, cifar_data_test, f"{model_name}_cifar_test", device)
predict_on_whole_dataset(model, cifar_data_train, f"{model_name}_cifar_train", device)
predict_on_whole_dataset(model, svhn_data_train, f"{model_name}_svhn_train", device)
predict_on_whole_dataset(model, svhn_data_test, f"{model_name}_svhn_test", device)

In [ ]:
model_name = "byol"

linear_model = train_linear_model(model_name=model_name, device=device, emb_size=2048)

In [ ]:
# torch.save(linear_model.state_dict(), f'../data/trained_models/{model_name}_cifar_linear_model0.905test_accuracy.pth')